In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [3]:
# %pip install faker

In [4]:
df = pd.read_excel('data/arquivo_anonimizado.xlsx')

In [5]:
faker = Faker('pt_BR')

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df['nome_completo'] = nomes_completos

In [6]:
df.head(5).T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 2817 non-null   int64  
 1   matriculaDRE               2817 non-null   int64  
 2   periodoIngressoUFRJ        2817 non-null   object 
 3   cursoIngressoUFRJ          2817 non-null   object 
 4   cursoAtual                 2817 non-null   object 
 5   periodoIngressoCursoAtual  2817 non-null   object 
 6   situacaoMatriculaAtual     2817 non-null   object 
 7   sexo                       2817 non-null   object 
 8   disciplinasCursadas        2650 non-null   object 
 9   crPorPeriodo               2644 non-null   object 
 10  craPorPeriodo              2642 non-null   object 
 11  periodosIntegralizados     2817 non-null   int64  
 12  cargaHorariaAcumulada      2817 non-null   int64  
 13  CRA                        2654 non-null   float

# Processamento

Algumas colunas não trazem ganho de informação, como por exemplo reprovações já estão na coluna disciplinas cursadas, e periodo com cr menor que 3, pode ser analisar na coluna CR por periodo

In [8]:
df.drop(['Unnamed: 0','CRA', 'periodosCRMenor3', 'periodosTrancados', 'periodosCancelados', 'reprovacoes'], axis=1, inplace=True)

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.

In [9]:
df.dropna(subset=['disciplinasCursadas'], inplace=True)

## Normaliazndo Colunas Multivaloradas

### CR por Periodo

In [10]:
df['crPorPeriodo'] = df['crPorPeriodo'].astype(str)
df['crPorPeriodo'] = df['crPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('crPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [11]:
df_crPeriodo = pd.DataFrame(df_exploded['crPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cr_periodo'))
df_crPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_crPeriodo

,periodo,cr_periodo,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.1,0
0,2001/1,5.9,0
0,2001/2,6.8,0
0,2002/1,6.3,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


### CRA por Periodo

In [12]:
df['craPorPeriodo'] = df['craPorPeriodo'].astype(str)
df['craPorPeriodo'] = df['craPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('craPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [13]:
df_craPeriodo = pd.DataFrame(df_exploded['craPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cr_acumulado'))
df_craPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_craPeriodo

,periodo,cr_acumulado,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.4,0
0,2001/1,6.9,0
0,2001/2,6.9,0
0,2002/1,6.8,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


### Disciplinas Cursadas

Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina

In [14]:
def processar_disciplinas(dados):
    dados = dados.astype(str)
    return dados.apply(lambda x: [split_disciplina(item) for item in x.split('\n')])

def split_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(' - ')
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = ' - '.join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)


In [15]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].astype(str)
# df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df['disciplinasCursadas'] = processar_disciplinas(df['disciplinasCursadas'])

In [16]:
df_exploded = df.explode('disciplinasCursadas')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas

In [17]:
df_exploded['disciplinasCursadas'] = df_exploded['disciplinasCursadas'].apply(lambda x: (
    x[0].replace(' ', ''),
    x[1].split()[0].replace(' ', ''),
    ' '.join(x[1].split()[1:]),
    x[2].replace(' ', ''),
    x[3].replace(' ', '')
))

In [18]:
df_disciplina = pd.DataFrame(df_exploded['disciplinasCursadas'].tolist(), index=df_exploded.index, columns=('periodo','cod_disciplina','disciplina','grau_disciplina','situacao_disciplina'))
df_disciplina['matriculaDRE'] = df_exploded['matriculaDRE']
df_disciplina

,periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


## Meger Colunas Tratadas

In [19]:
df_merge = pd.merge(df_disciplina, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge = pd.merge(df_merge, df_crPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge

,periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cr_acumulado,cr_periodo
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6
1,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0,7.6,7.6
2,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0,7.6,7.6
3,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0,7.6,7.6
4,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0,7.6,7.6
...,...,...,...,...,...,...,...,...
98634,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779,7.8,7.8
98635,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779,7.8,7.8
98636,2022/1,ICP120,Computação I (CC),075,Aprovado,2779,7.8,7.8
98637,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779,7.8,7.8


In [20]:
df_completo = pd.merge(df_merge, df, on=['matriculaDRE'], how='left')
df_completo.sample(5).T

,20618,42595,7689,60416,67044
periodo,2009/1,2008/1,2003/1,2017/1,2013/2
cod_disciplina,MAB230,FIT111,MAB232,MAB368,FIW125
disciplina,Calculo Numerico p/Informatica,Física I,Programação Linear I,Algoritmos e Grafos,"Mecânica, Oscilação e Ondas"
grau_disciplina,000,018,090,089,070
situacao_disciplina,Reprovadomedia,Reprovadomedia,Aprovado,Aprovado,Aprovado
matriculaDRE,480,978,180,1459,1628
cr_acumulado,4.8,6.2,7.3,6.7,6.9
cr_periodo,1.3,5.7,8.2,6.3,5.8
periodoIngressoUFRJ,2003/2,2007/2,2001/1,2011/1,2012/2
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bach em Ciências Matemáticas e da Terra,Bacharelado em Ciência da Computação


In [21]:
df_completo[['ano', 'semestre']] = df_completo['periodo'].str.split('/', expand=True)
df_completo.head(5).T

,0,1,2,3,4
periodo,2000/1,2000/1,2000/1,2000/1,2000/1
cod_disciplina,MAA123,MAB111,MAB120,MAE111,MAE115
disciplina,Algebra para Informatica,Fund da Computação Digital,Computacao para Informatica,Cálculo Infinitesimal I,Cálculo Vetorial e G Analitica
grau_disciplina,060,089,094,056,085
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matriculaDRE,0,0,0,0,0
cr_acumulado,7.6,7.6,7.6,7.6,7.6
cr_periodo,7.6,7.6,7.6,7.6,7.6
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática


In [22]:
df_completo.drop(['disciplinasCursadas', 'crPorPeriodo', 'craPorPeriodo'], axis=1, inplace=True)

### Padronizando Nomeclatura das Colunas

In [23]:
colunas = {
    'cod_disciplina': 'cod_disciplina',
    'disciplina': 'nome_disciplina',
    'matriculaDRE': 'matricula_dre',
    'periodoIngressoUFRJ': 'periodo_ingresso_ufrj',
    'cursoIngressoUFRJ': 'curso_ingresso_ufrj',
    'cursoAtual': 'curso_atual',
    'situacaoMatriculaAtual': 'situacao_matricula',
    'periodoIngressoCursoAtual': 'periodo_ingresso_curso_atual',
    'periodosIntegralizados': 'periodos_integralizados',
    'cargaHorariaAcumulada': 'carga_horaria_acumulada',
}
df_completo.rename(colunas, axis=1, inplace=True)

In [24]:
df_completo.rename(columns=colunas, inplace=True)

# Realizando Cagas das Tabelas

## Dimenssão Aluno

In [25]:
colunas_aluno = ['sk_d_aluno', 'nome_completo', 'matricula_dre', 'sexo', 'situacao_matricula']

In [26]:
d_aluno = df_completo[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matricula_dre', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matricula_dre,sexo,situacao_matricula
0,Maria Clara Fernandes,0,M,Cancelada por conclusão de curso
1,Lívia Martins,1,M,Cancelada por conclusão de curso
2,Ana Laura Araújo,2,M,Cancelada por abandono
3,Sra. Lívia Porto,3,M,Cancelada por conclusão de curso
4,Dra. Marcela Costela,4,M,Cancelada por conclusão de curso
...,...,...,...,...
2645,Dr. Davi Lucca Cavalcanti,2775,M,Ativa
2646,Giovanna Peixoto,2776,M,Ativa
2647,Srta. Mariane Dias,2777,M,Ativa
2648,Thales das Neves,2778,M,Ativa


In [27]:
d_aluno['sk_d_aluno'] = d_aluno.apply(lambda x: hashlib.sha256(str(x['matricula_dre']).encode()).hexdigest(), axis=1)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv('data/d_aluno.csv', index=False)
d_aluno

,sk_d_aluno,nome_completo,matricula_dre,sexo,situacao_matricula
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Maria Clara Fernandes,0,M,Cancelada por conclusão de curso
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Lívia Martins,1,M,Cancelada por conclusão de curso
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Ana Laura Araújo,2,M,Cancelada por abandono
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Sra. Lívia Porto,3,M,Cancelada por conclusão de curso
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Dra. Marcela Costela,4,M,Cancelada por conclusão de curso
...,...,...,...,...,...
2645,3da16474e33b330b71e5b562cfcf7939bde3c12a08b746...,Dr. Davi Lucca Cavalcanti,2775,M,Ativa
2646,9b3cea3bd8789ba47cc969bff5914da7ebac8e0052e282...,Giovanna Peixoto,2776,M,Ativa
2647,5a48eed290f62c93553855c36c964e1ef16603d23dcce3...,Srta. Mariane Dias,2777,M,Ativa
2648,a2541e84e912f4f5dd32ce131abdb6b1fdc9cf315790f1...,Thales das Neves,2778,M,Ativa


## Dimenssão Disciplina

In [28]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_completo[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [29]:
colunas_disciplina = ['sk_d_disciplina', 'cod_disciplina', 'nome_disciplina']

In [30]:
d_disciplina = df_completo[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_disciplina,nome_disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1589,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,MAE245,Álgebra Linear II - EQ
1592,ICP114,Computação I Ep


In [31]:
d_disciplina['sk_d_disciplina'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x['cod_disciplina'] + x['nome_disciplina']).encode()).hexdigest(), axis=1)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv('data/d_disciplina.csv', index=False)
d_disciplina

,sk_d_disciplina,cod_disciplina,nome_disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1589,f711fb63fc524f5758c111314b49b44d86a21e48bb458e...,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,eac458f82acdd92fd7aa393c4453804d5516adbbaa6158...,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,574c8aa6dc1f57bf5ca26827a0871cbd70ef3416288591...,MAE245,Álgebra Linear II - EQ
1592,9d40b24dedf530a6991c38a516e1275f152e62264a656e...,ICP114,Computação I Ep


## Dimenssão Periodo

In [32]:
colunas_periodo = ['sk_d_periodo', 'ano', 'semestre']

In [33]:
d_periodo = df_completo[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre
0,2000,1
1,2000,2
2,2001,1
3,2001,2
4,2002,1
...,...,...
67,2019,0
68,2021,0
69,2013,3
70,2014,3


In [34]:
d_periodo['sk_d_periodo'] = d_periodo.apply(lambda x: hashlib.sha256(str(x['ano'] + x['semestre']).encode()).hexdigest(), axis=1)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv('data/d_periodo.csv', index=False)
d_periodo

,sk_d_periodo,ano,semestre
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1
...,...,...,...
67,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0
68,fa122eae077795ac4ff0f80bb75550a9a2c30a192d035c...,2021,0
69,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3
70,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3


## Dimenssão Curso

In [35]:
colunas_curso = ['sk_d_curso', 'curso_ingresso_ufrj', 'curso_atual']

In [36]:
d_curso = df_completo[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,curso_ingresso_ufrj,curso_atual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Licenciatura em Matemática
3,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
73,Licenciatura em Física,Bacharelado em Ciência da Computação
74,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,Ciências Matemáticas e da Terra,Ciência da Computação
76,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [37]:
d_curso['sk_d_curso'] = d_curso.apply(lambda x: hashlib.sha256(str(x['curso_ingresso_ufrj'] + x['curso_atual']).encode()).hexdigest(), axis=1)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv('data/d_curso.csv', index=False)
d_curso

,sk_d_curso,curso_ingresso_ufrj,curso_atual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
3,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
73,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
74,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
76,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


## Dimenssão Situação Academica

In [276]:
# colunas_situacao_academica = ['sk_d_situacao_academica', 'situacaoMatriculaAtual']

In [277]:
# d_situacao_academica = df_completo[colunas_situacao_academica[1:]]
# d_situacao_academica = d_situacao_academica.drop_duplicates().reset_index(drop=True)
# d_situacao_academica

In [278]:
# d_situacao_academica['sk_d_situacao_academica'] = d_situacao_academica.apply(lambda x: hashlib.sha256(str(x['situacaoMatriculaAtual']).encode()).hexdigest(), axis=1)
# d_situacao_academica = d_situacao_academica.reindex(columns=colunas_situacao_academica)
# d_situacao_academica.to_csv('data/d_situacao_academica.csv', index=False)
# d_situacao_academica

## Fato Desempenho Academico

In [279]:
f_desempenho_academico = df_completo.drop(['periodo', 'periodo_ingresso_curso_atual', 'periodos_integralizados', 'carga_horaria_acumulada'], axis=1)
f_desempenho_academico.head(1)

,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,matricula_dre,cr_acumulado,cr_periodo,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,situacao_matricula,sexo,nome_completo,ano,semestre
0,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,Cancelada por conclusão de curso,M,Eduardo Sales,2000,1


In [280]:
# # D_SITUACAO_ACADEMICA
# f_desempenho_academico = pd.merge(f_desempenho_academico, d_situacao_academica, on=['situacao_matricula_atual'], how='left')
# f_desempenho_academico.drop(['situacaoMatriculaAtual'], axis=1, inplace=True)
# f_desempenho_academico.sample(5).T

In [281]:
# D_PERIODO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_periodo, on=['ano', 'semestre'], how='left')
f_desempenho_academico.drop(['ano', 'semestre'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,34275,49609,62972,7720,30183
cod_disciplina,MAB120,MAB489,MAB510,MAE115,MAB120
nome_disciplina,Computacao para Informatica,Banco de Dados I,Teleprocessamento e Redes,Cálculo Vetorial e G Analitica,Computacao para Informatica
grau_disciplina,061,012,062,095,059
situacao_disciplina,Aprovado,Reprovadomedia,Aprovado,Aprovado,Aprovado
matricula_dre,788,1140,1524,181,694
cr_acumulado,5,5.3,6.4,8.7,3.2
cr_periodo,5,3.6,7.7,8.7,3.2
periodo_ingresso_ufrj,2006/1,2008/2,2011/2,2001/1,2005/1
curso_ingresso_ufrj,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação
curso_atual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [282]:
# D_CURSO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_curso, on=['curso_ingresso_ufrj', 'curso_atual'], how='left')
f_desempenho_academico.drop(['curso_ingresso_ufrj', 'curso_atual'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,51805,50504,62509,73790,48376
cod_disciplina,MAB624,MAE115,MAB122,MAB123,FIW230
nome_disciplina,Números Inteiros Criptografia,Cálculo Vetorial e G Analitica,Computação Gráfica I,Linguagens Formais,Eletromagnetismo e Ótica
grau_disciplina,000,000,030,097,039
situacao_disciplina,Reprovadomedia,Reprfalta/media,Reprovadomedia,Aprovado,Reprovadomedia
matricula_dre,1200,1164,1514,1799,1107
cr_acumulado,2.7,0.5,5.4,9.3,5.6
cr_periodo,2.7,0,2.6,9.4,4.1
periodo_ingresso_ufrj,2009/1,2009/1,2011/2,2014/1,2008/2
situacao_matricula,Cancelada por abandono Definitivo,Cancelamento por opção de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,F,M,M,F


In [283]:
# D_ALUNO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_aluno, on=['nome_completo', 'sexo', 'matricula_dre', 'situacao_matricula'], how='left')
f_desempenho_academico.drop(['nome_completo', 'sexo', 'matricula_dre','situacao_matricula'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,35244,51082,34273,93009,59488
cod_disciplina,MABX02,MABX01,MABX02,ICP111,MAB354
nome_disciplina,Projeto Final de Curso,Atividades Complementares,Projeto Final de Curso,Fund Computação Digital,Computadores e Sociedade
grau_disciplina,080,,100,068,087
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
cr_acumulado,4.9,8.2,9.2,5.1,6.1
cr_periodo,5,3.6,10,6.1,6.6
periodo_ingresso_ufrj,2006/1,2009/1,2006/1,2019/1,2011/1
sk_d_periodo,12ae265fcfe315a363b94cb06b5634257af6517781d59a...,e54a2041a494a6534e33f777b64c7912f8a2cc5a4e9a33...,b18c61c55f17af492ac0535bf17d89dfb521c9110d0cbd...,3690477c088dfac6646dadb816d998c06a6919f781bb01...,d199c5cf655fc67ae5bc24ba22fd2e197e3a28f3794760...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,69f98f9a931103c9534d1881425cd98f8d920686439325...
sk_d_aluno,5d85be4cc5af40a7cf2c4f0818d92689c185fdea656674...,2c58b3a68ac99f845a207a613ce245b3fd2dd839320355...,fc091d39524c9d4b5b11f84f9132996a94ca01c9816d2d...,22492061879a16b906982f2381ad3b4091282f6ed1cbcc...,a4ff3ad278c7b0575a8e32ae08eebfb327e37762cb1a2c...


In [284]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(f_desempenho_academico, d_disciplina, on=['cod_disciplina', 'nome_disciplina'], how='left')
f_desempenho_academico.drop(['cod_disciplina', 'nome_disciplina'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,31222,95504,69677,98492,7805
grau_disciplina,084,072,063,081,039
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Reprovadomedia
cr_acumulado,8.6,5.6,6.5,8.4,3
cr_periodo,7.8,6.6,6.9,8.4,1.9
periodo_ingresso_ufrj,2005/1,2019/2,2013/1,2022/1,2001/1
sk_d_periodo,9652f0351390fbe354fe9370bc85f99e126751f682ccfd...,1d1608805160ee7f833c28de50dd6887e8910ae2a7418f...,9bbebbaaa8b874af4d5c1a4083cc49bfd3a98a897cd1c0...,5168f166ecaf66e62ce5c859e54e06e615b86260677db2...,c2e8c4d907eb354cddc82e7ae2cfeb2f4fe821c08a8c10...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,a80f5dd573274667573b525e0750e7915a3bc967190075...,a80f5dd573274667573b525e0750e7915a3bc967190075...,4731a731e22171cf552cc83fe54a18bda1ce0bb144efa1...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...
sk_d_aluno,35c71bd7eaf4607047bb7c186d17251942204229b897e0...,72e37da8a67d137ab41fe3a0ef269ad619192eb0d93a4a...,a8caa602fb66f78027e2035b4200e304e21bb21b88ff6c...,bd64e1346049830018c72cd3623c277fc5095c34bbb07d...,b8aed072d29403ece56ae9641638ddd50d420f950bde0e...
sk_d_disciplina,6eb06967153688b4bdf3fddbf9c9002e60d0cd79638226...,1a80b254db710778a5a7610c938ba5a877503cba21e0f9...,f1787f79ef198d2337ead7a2287c52aa6976f0fb53b38b...,e940a1ccfa4a04e20701b54df428417d6604e89431f800...,a790a2f3d8c80fd7e382113da81d41fa6c1720185bf086...


In [285]:
f_desempenho_academico.drop(['periodo_ingresso_ufrj'], axis=1, inplace=True)

In [286]:
f_desempenho_academico.sample(5).T

,46693,21477,68730,95429,97254
grau_disciplina,T,027,000,071,100
situacao_disciplina,Aprovado,Reprovadomedia,Reprovadomedia,Aprovado,Aprovado
cr_acumulado,NaN,6.1,0.7,6.6,9.5
cr_periodo,NaN,5.4,0.7,6.5,9.5
sk_d_periodo,da6418b9c3521c3a2dbcc99c04df921ca9df29ad856e6f...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...,e319d9b68ca50f52f6dfba34b7784ba81c72646766f52c...,df37d0648b2961147eeca17915e97689087f5feefa44f0...,a410ca8badb0e813980534c7b60a5425432d4ff5921a13...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,a80f5dd573274667573b525e0750e7915a3bc967190075...,a80f5dd573274667573b525e0750e7915a3bc967190075...
sk_d_aluno,f48280c0107726b99afcbd3cd0a46a2bac5f1ddb44e876...,db3defda18fafc0c197740438051c690d98b551a7e449d...,5514a9f709310b22ee9bddd4e6da1b2b8b04d1ad5c3dcb...,dc7d6256d124bcc9244d6d0e437beabded4b25d4fbf77c...,332d88d62030ed158213fa33b60f3d03a4d2cfccf9a64c...
sk_d_disciplina,de4f94b5804ebaccfa8f5d68972cfbd7693a996d28f224...,de4f94b5804ebaccfa8f5d68972cfbd7693a996d28f224...,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,e06a0c438f2d90d5bcd5bab6dd498c4db684593c35a3d2...


In [287]:
f_desempenho_academico.to_csv('data/f_desempenho_academico.csv', index=False)

# Analise de disciplinas com nomes errados

In [288]:
d_disciplina[d_disciplina['cod_disciplina'] == 'MAB613']

,sk_d_disciplina,cod_disciplina,nome_disciplina
51,69c74b0dc4bd99f8f39728f26b5edb757d2f12f54e08ea...,MAB613,Met Desenv de Sist Informacao
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação


In [289]:
d_disciplina[d_disciplina['cod_disciplina'] == 'MAB601']
# EEL410

,sk_d_disciplina,cod_disciplina,nome_disciplina
50,fa1851c7178a5ca652bdbb609397732777b444e5be4aa4...,MAB601,Top Esp em Eng de Softaware
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software


In [290]:
d_disciplina[d_disciplina['cod_disciplina'].duplicated()]

,sk_d_disciplina,cod_disciplina,nome_disciplina
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação
358,bb19397c608b8ad6e1e91f353d8aeaa3c0109a13a33ecb...,MAB603,Gestão do Conhecimento
383,58ae4c22500c1bf1a3dd156b6405367f685508d5a856ab...,MAB230,Cálculo Numérico (CC)
417,70c5438681f6efe2902a4b9e78fd4488412a85544fdf14...,MAB602,Dw no Sup à Tomada de Decisão
419,c6fdafe7054dc9166dd654d7d98ef1d8da16718f40133a...,MAB600,Dados Semi-estruturados e XML
460,ad46eeb3845a2b7de8e5ceebc4230a1850be2509cad643...,MAB532,Mineração de Dados
462,a8cff0b3f3cdc6991275d971f5fef3fdfbbf8d6393afbf...,EEH210,Engenharia e Meio Ambiente
471,6350703008dfb26f834d42c1683bf89f50315ddbea870e...,IEE304,Desenvolvimento Socioeconomico
474,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,MAB120,Computação I (CC)
